In [1]:
import numpy as np
import matplotlib.pyplot as plt
import evaluate
from mpl_toolkits.mplot3d import Axes3D
import gymnasium
from run.train import configuration
import json
from copy import copy, deepcopy
import panda_gym
import pybullet
from sb3_contrib import TQC
import winsound

C:\Users\eclip\GitHub\panda-gym\venv\Lib\site-packages\gymnasium-0.28.1-py3.11.egg\gymnasium\envs\registration.py:693: UserWarning: WARN: Overriding environment PandaReachAO-v3 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [2]:
# standard settings
human=False
reward_type="kumar_her"
n_substeps=20
max_ep_steps = 200
goal_condition="reach"
#register envs
panda_gym.register_reach_ao(max_ep_steps)

# visual stuff
sphere_list = []
sphere_count = 0

# narrow_tunnel
evaluation_scenario="narrow_tunnel"
prior_orientation="left"

# library 2
evaluation_scenario="library2"
prior_orientation="back"

In [3]:
# workshop
evaluation_scenario="workshop"
prior_orientation="fkine"

# wall
evaluation_scenario="wall"
prior_orientation="fkine"

In [4]:
# get work env
work_env = gymnasium.make(configuration["env_name"], render=False, control_type="js",
                     obs_type=configuration["obs_type"], goal_distance_threshold=0.05,
                     goal_condition=goal_condition,
                     reward_type=reward_type, limiter=configuration["limiter"],
                     show_goal_space=False, scenario=evaluation_scenario,
                     randomize_robot_pose=False,  # if evaluation_scenario != "wang_3" else True,
                     joint_obstacle_observation="vectors+all",
                     truncate_on_collision=True,
                     terminate_on_success=True,
                     show_debug_labels=True, n_substeps=n_substeps)


In [5]:
work_env.robot.control_type = "jsd"
results, metrics = evaluate.evaluate_ensemble(["prior"], work_env, human=human, num_episodes=100, deterministic=True,
                                     strategy="",
                                     scenario_name=evaluation_scenario,
                                     prior_orientation=prior_orientation)
ee_pos = metrics["end_effector_positions"]
path_success = metrics["path_success"]
done_events  = metrics["done_events"]

workshop:   0%|          | 0/100 [00:00<?, ?it/s]

Restricted license - for non-production use only - expires 2024-10-28


C:\Users\eclip\GitHub\panda-gym\panda_gym\envs\robots\panda.py:375: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if target_pose == "left":
C:\Users\eclip\GitHub\panda-gym\panda_gym\envs\robots\panda.py:377: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif target_pose == "right":
C:\Users\eclip\GitHub\panda-gym\panda_gym\envs\robots\panda.py:379: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif target_pose == "front":
C:\Users\eclip\GitHub\panda-gym\panda_gym\envs\robots\panda.py:381: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif target_pose == "back":
C:\Users\eclip\GitHub\panda-gym\panda_gym\envs\robots\panda.py:383: FutureWarning: elementwise compariso

In [10]:
work_env.robot.control_type = "js"
model_name = "glad-universe-418"
model = TQC.load(fr"../run/run_data/wandb/{model_name}/files/best_model.zip", env=work_env,
                 custom_objects={"action_space": gymnasium.spaces.Box(-1.0, 1.0, shape=(7,),
                                                                      dtype=np.float32)})  # for some reason it won't read action space sometimes

results, metrics = evaluate.evaluate_ensemble([model], work_env, human=human, num_episodes=100, deterministic=True,
                                     strategy="variance_only",
                                     scenario_name=evaluation_scenario,
                                     prior_orientation="")
ee_pos = metrics["end_effector_positions"]
path_success = metrics["path_success"]
done_events  = metrics["done_events"]

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


workshop: 100%|██████████| 100/100 [04:23<00:00,  2.63s/it]


In [ ]:
# get rendered env
env = gymnasium.make(configuration["env_name"],
                     render=True, control_type="js",
                     obs_type=configuration["obs_type"], goal_distance_threshold=0.05,
                     goal_condition="halt",
                     reward_type=reward_type, limiter=configuration["limiter"],
                     show_goal_space=True, scenario=evaluation_scenario,
                     randomize_robot_pose=False,  # if evaluation_scenario != "wang_3" else True,
                     joint_obstacle_observation="vectors+all",
                     truncate_on_collision=True,
                     terminate_on_success=True,
                     show_debug_labels=True, n_substeps=n_substeps)

In [11]:
# clear env
env.task.sim.remove_all_debug_text()
for sphere in sphere_list:
    env.task.sim.physics_client.removeBody(env.task.sim._bodies_idx[sphere])
env.task.create_goal_outline() # workaround

# hide goal
env.task.sim.set_base_pose("target", np.array([0.0, 0.0, -1.0]), np.array([0.0, 0.0, 0.0, 1.0]))
# change visual shape of robot

path_suc2 = path_success[:50]
done_suc2 = np.ones(50) #metrics["done_events"][100:200]
done_normal = done_events[100:200]
path_normal = ee_pos[100:200]

for trajectory, done_event in zip(ee_pos, done_events):
    color = 1.0
    traj = deepcopy(trajectory)
    final_pos = copy(traj[-1])
    xyz1 = traj.pop()


    while traj:
        xyz2 = traj.pop()
        pybullet.addUserDebugLine(lineFromXYZ=xyz1, lineToXYZ=xyz2, lineColorRGB=np.array([1-color,color,0]), physicsClientId=1, lifeTime=0, lineWidth=2)
        color = len(traj)/len(trajectory)#1/max_ep_steps
        xyz1 = xyz2

    if done_event == 1:
            env.task.sim.create_sphere(
            body_name=f"sphere{sphere_count}",
            radius=0.01,
            mass=0.0,
            ghost=True,
            position=final_pos,
            rgba_color=np.array([0.0, 1.0, 0.0, 1.0]),
        )
    elif done_event == 0:
        env.task.sim.create_sphere(
            body_name=f"sphere{sphere_count}",
            radius=0.01,
            mass=0.0,
            ghost=True,
            position=final_pos,
            rgba_color=np.array([1.0, 1.0, 0.0, 1.0]),
        )
    else:
        env.task.sim.create_sphere(
            body_name=f"sphere{sphere_count}",
            radius=0.01,
            mass=0.0,
            ghost=True,
            position=final_pos,
            rgba_color=np.array([1.0, 0.0, 0.0, 1.0]),
        )
    sphere_list.append(f"sphere{sphere_count}")
    sphere_count += 1

# plot with debug line


# Plotting
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
#
# ax.plot(x, y, z, label='3D trajectory')
# ax.legend()
#
# plt.show()
winsound.Beep(2000, 1000)

In [9]:

pybullet.changeVisualShape(objectUniqueId=0, linkIndex=2, rgbaColor=[1,1,1,0.5], physicsClientId=1)
pybullet.changeVisualShape(objectUniqueId=0, linkIndex=3, rgbaColor=[1,1,1,0.5], physicsClientId=1)
# pybullet.changeVisualShape(objectUniqueId=0, linkIndex=2, rgbaColor=[1,1,1,0.5])
# pybullet.changeVisualShape(objectUniqueId=0, linkIndex=1, rgbaColor=[1,1,1,0.5])
# pybullet.changeVisualShape(objectUniqueId=0, linkIndex=0, rgbaColor=[1,1,1,0.5])